### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2018-10-05 17:16:32--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Распознаётся www.dropbox.com (www.dropbox.com)… 2620:100:6022:1::a27d:4201, 162.125.66.1
Подключение к www.dropbox.com (www.dropbox.com)|2620:100:6022:1::a27d:4201|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [переход]
--2018-10-05 17:16:33--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Повторное использование соединения с [www.dropbox.com]:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc33b9ba38b8087a9bb57e76583b.dl.dropboxusercontent.com/cd/0/get/ASP-17MTT0avp8I9lssJkyPqpYhIL-IVxXmIi_iSsuZHEc_PxWRzTpolTluMQ0ag8oXOdZ6IWvVTUlXrWzDol1oeFFtCg5RIGenZ6uLkRsKMCU5nMLkqAX9oRJNvULOqJCsnQG7M3SnPNCvvGT0g_Sdh8_4nZ2lpvZ2S-j4fwWKzWU0exoTodSVMoMOjJKz8lpc/file?dl=1 [переход]
--2018-10-05 17:16:33--  https://uc33b9ba38b8087a9bb57e76583b.dl.dropboxusercontent.com/cd

,author,day,id,link,month,summary,tag,title,year
7364,"[{'name': 'Yiren Zhou'}, {'name': 'Seyed-Mohse...",4,1712.01048v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,In recent years Deep Neural Networks (DNNs) ha...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Adaptive Quantization for Deep Neural Network,2017
7720,"[{'name': 'Jeremy Aghaei Mazaheri'}, {'name': ...",28,1802.10497v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Sparse representations using overcomplete dict...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Learning Discriminative Multilevel Structured ...,2018
13372,"[{'name': 'P. A. Castillo'}, {'name': 'M. G. A...",25,1105.4971v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,This paper analises distributed evolutionary c...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",Distributed Evolutionary Computation using REST,2011
20927,"[{'name': 'Katsunari Shibata'}, {'name': 'Yuki...",16,1705.05551v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Expectation for the emergence of higher functi...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",New Reinforcement Learning Using a Chaotic Neu...,2017
7200,"[{'name': 'Laura Douglas'}, {'name': 'Iliyan Z...",2,1711.00695v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,We consider the problem of inference in a caus...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",A Universal Marginalizer for Amortized Inferen...,2017


In [4]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [5]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

#tokenize_sents
lines = [' '.join(tokenizer.tokenize(line.lower())) for line in lines]


In [6]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_{t + 1} \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [7]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        line = (UNK + ' ')* (n - 1) + line + ' ' + EOS
        words = line.split()
        for i in range(n - 1, len(words)):
            counts[tuple(words[i - n + 1: i])][words[i]] += 1
    
    return counts


In [8]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [9]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prev_words in counts.keys():
            count_prefix = 0
            for word in counts[prev_words].keys():
                count_prefix += counts[prev_words][word]
            for word in counts[prev_words].keys():
                self.probs[prev_words][word] = counts[prev_words][word] / count_prefix
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [10]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [11]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [11]:
token_probs = lm.get_possible_next_tokens('neural')
sorted(token_probs.keys(), key=token_probs.get, reverse=True)

['network',
 'networks',
 'machine',
 '-',
 'semantic',
 'architectures',
 'multi',
 'text',
 'variational',
 'program',
 'models',
 'question',
 'architecture',
 'symbolic',
 'style',
 'discourse',
 'system',
 'turing',
 'reranking',
 'net',
 'word',
 'paraphrase',
 'lattice',
 'enquirer',
 'associative',
 'module',
 'photo',
 'responding',
 'domain',
 'speech',
 'tuning',
 'taylor',
 'expectation',
 'component',
 'belief',
 'programmer',
 'tree',
 'probabilistic',
 'optimizer',
 'coarse',
 'combinatorial',
 'block',
 'crf',
 'realisation',
 'aggregation',
 'stain',
 'dynamic',
 'random',
 'decision',
 'episodic',
 'embeddings',
 'sequence',
 'relational',
 'conditional',
 'affine',
 '3d',
 'algebra',
 'rating',
 'self',
 'ranking',
 'voice',
 'audio',
 'slam',
 'task',
 'hypernetwork',
 'nilm',
 'decomposition',
 'distributed',
 'attention',
 'headline',
 'document',
 'emoji',
 'generative',
 'summarization',
 'recovery',
 'morphological',
 'name',
 'sentence',
 'versus',
 'structura

In [12]:
import scipy.stats as sps
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ temperature
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    token_probs = lm.get_possible_next_tokens(prefix)
    if temperature == 0:
        return np.array(list(token_probs.keys()))[np.argmax(list(token_probs.values()))]
    
    sum_probs = np.sum(list(token_probs.values()))
    new_probs = np.array(list(token_probs.values())) ** (1 / temperature)
    token = np.random.choice(list(token_probs.keys()), size=1,  p=new_probs / new_probs.sum())
    return token[0]


In [13]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [14]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial intelligence , which interleaves efficient model - based algorithms to improve the quality of the well - defined parameters , which are then used in various clinical informatics tasks . _EOS_


In [15]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between theory and applications ; recent advances in deep learning ; in this paper , we show that the proposed method is shown that the proposed method is effective in practice . _EOS_


### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [16]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    log_perplexity = 0
    count_words = 0
    for line in lines:
        sent = line.split()
        prefix = [UNK] * (lm.n - 1)
        prob = lm.get_next_token_prob(' '.join(prefix), sent[0])
        if prob > 0:
            addition = np.log(prob)
            if addition < min_logprob:
                addition = min_logprob
            log_perplexity += addition
        else:
            log_perplexity += min_logprob

        count_words += len(sent) + 1
        for i, token in enumerate(sent[:-1]):
            prefix = prefix[1:] + [token]
            prob = lm.get_next_token_prob(' '.join(prefix), sent[i + 1])
            if prob > 0:
                addition = np.log(prob)
                if addition < min_logprob:
                    addition = min_logprob
                log_perplexity += addition
            else:
                log_perplexity += min_logprob
            
        prefix = prefix[1:] + [sent[-1]]
        prob = lm.get_next_token_prob(' '.join(prefix), EOS)
        if prob > 0:
            addition = np.log(prob)
            if addition < min_logprob:
                addition = min_logprob
            log_perplexity += addition
        else:
            log_perplexity += min_logprob
            
    return np.exp(-log_perplexity / count_words)

In [17]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
print(ppx1, ppx3, ppx10)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

318.2132342216302 1.5199996213739575 1.1838145037901249
Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [31]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259043346743296.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distrivution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [18]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [19]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [166]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [175]:
lm = LaplaceLanguageModel(train_lines, n=2, delta=0.1)
prefix = 'ability'
for i in range(10):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

ability pronet wedescribe ikea countermeasures mte yin explanations theories bilateral tributes


### Kneser-Ney smoothing

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} \max(0, Count(prefix_{n-1}, \hat w) - \delta)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within one order of magnitude) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [26]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        self.lines
        self.delta
        self.probs = self.evaluate_probs()

    def evaluate_probs(self, cur_n):
        counts = count_ngrams(self.lines, cur_n)
        probs = defaultdict(Counter)
        if cur_n == 1:
            #compute N_1+
            unique_prev_words = {}
            for line in self.lines:
                line = UNK + ' ' + line + ' ' + EOS
                words = line.split()
                for i in range(1, len(words)):
                    if words[i] not in unique_prev_words.keys():
                        unique_prev_words[words[i]] = set()
                    unique_prev_words[words[i]].add(words[i-1])
            sum_counts_unique = np.sum([len(w) for w in unique_prev_words.values()])
            for word in unique_prev_words.keys(): 
                probs[tuple()][word] = len(unique_prev_words[word]) / sum_counts_unique
        else:
            for prefix in counts.keys():
                count_prefix = np.sum(list(counts[prefix].values()))
                for word in counts[prefix].keys():
                    probs[prefix][word] = (max(count[prefix][word] - self.delta, 0)  + 
                                        self.delta * len(count[prefix].keys()) * count_probs(cur_n - 1)) / count_prefix
        return probs
                
                
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
        
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [ ]:
self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}

In [27]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [38]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.5)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 976.68126
N = 2, Perplexity = 908.44743
N = 3, Perplexity = 1671.85843


In [35]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial odc softwares contraption existed convenience ucpop maxi enought kover immensely negentropy ahu ference eva ccg segmenters hbci lpnmr marketplaces javanmard toddlers hemispheric charac nondegenerate dvi protected rsa eric accepts overhead regularly textipa universe disruption modèle semiconducting guarico rethinking paint reweigh controls lennard clusterized sandboxing predicament hydrographic evolution hatred visualtracking filesystem osmosis reorganization judgmental diploglossus syntactically eld earch inelegant inflation focusnet elevated skirmishes cruces telephony wholes pdl destabilize idiom hypercliques eyewitness trysts monkeytyping sparc disputed stations administrator lorentzian hkust headphones braverman wordnet1 jia rho_ sentencing cisor postures sparfa cles haystack déterminants newcomb matter lightside bioimage cmtf orie pieces greeting empirical contributors
